In [ ]:
import torch
from torch import nn
from torchvision.models import resnet18, ResNet18_Weights, resnet50, ResNet50_Weights

# resnet50 = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

S = 7
C = 20

resnet18 = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
backbone = nn.Sequential(*list(resnet18.children())[:-2])  # 去掉最后两层

In [ ]:
head=nn.Sequential(
    nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=3,padding=1), # (batch,1024,14,14)
    nn.LeakyReLU(0.1),
    nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,padding=1),
    nn.LeakyReLU(0.1),
    nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,padding=1), # (batch,1024,7,7)
    nn.LeakyReLU(0.1),

    nn.Flatten(),
    nn.Linear(in_features=S*S*4096,out_features=4096),
    nn.Dropout(),
    nn.LeakyReLU(0.1),
    nn.Linear(in_features=4096,out_features=S*S*(10+C)),
    nn.Sigmoid()
)

In [ ]:
input = torch.randn(1,3,448,448)
output = backbone(input)
output = head(output)

output.shape

In [ ]:
from torch.utils.data.dataloader import DataLoader
from datasets import YoloVOCDataset

dataloader = DataLoader(YoloVOCDataset(448, 7, 20), shuffle=True)

for batch_x,batch_y in dataloader:
    batch_x,batch_y=batch_x.to('cpu'),batch_y.to('cpu')